# Experiment 3

Compare error rates on **simulated data** for the following:

- No coding (trace reconstruction): BMALA improved vs Trellis BMA without lookahead vs Trellis BMA with lookahead
- Coded trace reconstruction: Trellis BMA without lookahead vs Trellis BMA with lookahead

In [5]:
from helper_functions import *
import pandas as pd
import numpy as np
from scipy.stats import mode

from Levenshtein import distance, editops
from scipy.stats import mode
from tqdm import trange

from conv_code import *
from coded_ids_multiD import *
from bma import *
from trellis_bma import *

import time

import multiprocessing as mp
from multiprocessing.pool import ThreadPool as Pool
from one_iter import one_iter_rest
from itertools import repeat
# import seaborn as sns
# sns.set()

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def make_plot_template(plot_type = "Hamming rate", xlim = (0,10), ylim = (0,50), figsize = (6,6)):
    import matplotlib 
    matplotlib.rc('xtick', labelsize=14) 
    matplotlib.rc('ytick', labelsize=14) 
    
    plt.figure(figsize = figsize)
    plt.xlabel("Number of traces",fontsize = 14)
    
    if plot_type == "Hamming rate":
        plt.ylabel("Normalized Hamming error rate",fontsize = 14)
    
    elif plot_type == "Information rate":
        plt.ylabel("Information rate (in bits/base)",fontsize = 14)
    
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.minorticks_on()
    plt.grid(b=True, which='major', linestyle='-')
    plt.grid(b=True, which='minor', linestyle='--',linewidth = 0.5)
    
    

def add_plot(data, N, keys, labels, plot_type = "Hamming rate", markers = None):
    
    if markers is None:
        markers = ["*","o","v","^","s","p","<",">","1","2","3","4","8","s","p","P",".",",","h",\
                   "H","+","x","X","D","d","|","_",0,1,2,3,4,5,6,7,8,9,10,11]
    
    for idx, key in enumerate(keys):
        if plot_type == "Hamming rate":
            plt.plot(data["cluster_size"],data[key].mean(axis=1)/N, label = label[key], linewidth = 2, marker = markers[idx])
        elif plot_type == "Information rate":
            plt.plot(data["cluster_size"],data[key].mean(axis=1), label = label[key], linewidth = 2, marker = markers[idx])
    
    plt.legend(fontsize = 14)

## Uncoded error rates

In [ ]:
in_len = 110
N_cw = 110
redundancy = N_cw-in_len

A_in = 4
A_cw = 4

cc = conv_code()
G = np.array([[1]])
cc.quar_cc(G)
cc.make_trellis(in_len)
# cc.puncture(redundancy=redundancy)
# cc.add_coset()
# cc.make_encoder()
code_trellis_states = cc.trellis_states
code_trellis_edges = cc.trellis_edges
code_time_type = cc.time_type

num_traces = 1
p_del = 0.02      
p_sub = 0.022
p_ins = 0.017
max_drift = 6

ids_trellis = coded_ids_multiD(A_in, A_cw, code_trellis_states,code_trellis_edges, code_time_type,\
                 num_traces, p_del, p_sub, p_ins, max_drift, input_prior = None)

### MultiD

In [ ]:
# Real data
max_iters = 1000

errors = {}
errors["desc"] = "Experiment comparing error rates and IRs on simulated data without coding. \n\
Algorithms compared: MultiD bcjr.\n\
"

errors["multiD"] = []
errors["multiD_AIRs"] = []
errors["multiD_BORs"] = []
errors["cluster_size"] = [2,3]

for cluster_size in errors["cluster_size"]:
    if cluster_size == 2:
        max_drift = 10
    elif cluster_size == 3:
        max_drift = 6
    elif cluster_size == 4:
        max_drift = 4
    
    ids_trellis = coded_ids_multiD(A_in, A_cw, code_trellis_states,code_trellis_edges, code_time_type,\
                     cluster_size, p_del, p_sub, p_ins, max_drift, input_prior = None)

    multiD_errors = []
    multiD_AIRs = []
    multiD_BORs = []
    
    for it in trange(max_iters):
        in_seq = np.random.choice(4,size = in_len)
        
        tr_list = []
        for j in range(cluster_size):
            tr_list.append(ids_trellis.simulate_ids(in_seq))
            
        traces = []
        for tr in tr_list:
            if np.abs(len(tr)-N_cw) <= max_drift:
                traces.append(tr)
            elif len(tr) > N_cw:
                idx_delete = np.random.choice(len(tr),len(tr)-N_cw-max_drift, replace = False)
                traces.append(np.delete(tr,idx_delete))
            else:
                idx_insert = np.random.choice(len(tr),N_cw-max_drift-len(tr), replace = False)
                traces.append(np.insert(tr,idx_insert,0))
                
        post_probs = ids_trellis.bcjr(traces,cc.trellis_states[0][0],cc.trellis_states[-1])
        multiD_estimate = post_probs.argmax(axis=1)
        multiD_errors.append((multiD_estimate != in_seq).sum())
        
        logp_y = ids_trellis.logp_y * 1.0
        P = np.zeros((in_len,4))
        P[np.arange(in_len),in_seq] = 1
        ids_trellis.bcjr(traces,cc.trellis_states[0][0],cc.trellis_states[-1], input_prior = P)
        logp_ylx = ids_trellis.logp_y * 1.0
        
        if (logp_ylx-logp_y)/in_len == -np.inf:
            raise ValueError("Infinity")
        
        multiD_AIRs.append((logp_ylx-logp_y)/in_len)
        multiD_BORs.append(BCJROR(post_probs, in_seq))
        
        
    errors["multiD"].append(np.array(multiD_errors))
    errors["multiD_AIRs"].append(np.array(multiD_AIRs))
    errors["multiD_BORs"].append(np.array(multiD_BORs))
    
    print("MultiD error rate for a cluster size: ",cluster_size,"is ",np.array(multiD_errors).mean())
    print("MultiD AIRs for a cluster size: ",cluster_size,"is ",np.array(multiD_AIRs).mean())
    print("MultiD BORs for a cluster size: ",cluster_size,"is ",np.array(multiD_BORs).mean())
    
    time.sleep(0.5)

errors["multiD"] = np.array(errors["multiD"])
errors["multiD_AIRs"] = np.array(errors["multiD_AIRs"])
errors["multiD_BORs"] = np.array(errors["multiD_BORs"])

np.save("SavedData/sim_uncoded_multiD.npy",errors)

### Trellis BMA and others

In [9]:
in_len = 110
N_cw = 110
redundancy = N_cw-in_len

A_in = 4
A_cw = 4

cc = conv_code()
G = np.array([[1]])
cc.quar_cc(G)
cc.make_trellis(in_len)
# cc.puncture(redundancy=redundancy)
# cc.add_coset()
# cc.make_encoder()
code_trellis_states = cc.trellis_states
code_trellis_edges = cc.trellis_edges
code_time_type = cc.time_type

num_traces = 1
p_del = 0.02      
p_sub = 0.022
p_ins = 0.017
max_drift = 15

ids_trellis = coded_ids_multiD(A_in, A_cw, code_trellis_states,code_trellis_edges, code_time_type,\
                 num_traces, p_del, p_sub, p_ins, max_drift, input_prior = None)

In [10]:
max_iters = 100

errors = {}
errors["desc"] = "Experiment comparing error rates and IRs on simulated data without coding. \n\
Algorithms compared: Improved BMALA, Post multiply, Trellis BMA no look-ahead, Trellis BMA with look-ahead.\n\
"

errors["bma"] = []
errors["post_mul"] = []
errors["BOR_post_mul"] = []
errors["Tbma_noLA"] = []
errors["Tbma_LA"] = []
errors["BOR_Tbma_noLA"] = []
errors["BOR_Tbma_LA"] = []
errors["cluster_size"] = [2,3,10]

for cluster_size in errors["cluster_size"]:
    bma_errors = []
    post_mul_errors = []
    post_mul_BORs = []
    Tbma_noLA_errors = []
    Tbma_LA_errors = []
    Tbma_noLA_BORs = []
    Tbma_LA_BORs = []
    
    for it in trange(max_iters):
#         if len(traces_list[it]) == 0:
#             print("Encountered empty cluster, ignored it.")
        
        in_seq = np.random.choice(4,size = in_len)
        
        tr_list = []
        for j in range(cluster_size):
            tr_list.append(ids_trellis.simulate_ids(in_seq))
            
        traces = []
        for tr in tr_list:
            if np.abs(len(tr)-N_cw) <= max_drift:
                traces.append(tr)
            elif len(tr) > N_cw:
                idx_delete = np.random.choice(len(tr),len(tr)-N_cw-max_drift, replace = False)
                traces.append(np.delete(tr,idx_delete))
            else:
                idx_insert = np.random.choice(len(tr),N_cw-max_drift-len(tr), replace = False)
                traces.append(np.insert(tr,idx_insert,0))
        
        bma_estimate = bmala_imp(N_cw,tr_list,2)
        
        post_mul_estimate, post_mul_probs = post_multiply(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1])
        
#         Tbma_noLA_estimate, Tbma_noLA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
#                                                  cc.trellis_states[-1],lookahead = False)
        Tbma_LA_estimate, Tbma_LA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1],lookahead = True)
        
        bma_errors.append((in_seq != bma_estimate).sum())
        
        post_mul_errors.append((in_seq != post_mul_estimate).sum())
        post_mul_BORs.append(BCJROR(post_mul_probs,in_seq))
        
#         Tbma_noLA_errors.append((in_seq != Tbma_noLA_estimate).sum())
        Tbma_LA_errors.append((in_seq != Tbma_LA_estimate).sum())
#         Tbma_noLA_BORs.append(BCJROR(Tbma_noLA_probs, in_seq))
        Tbma_LA_BORs.append(BCJROR(Tbma_LA_probs, in_seq))

    errors["bma"].append(np.array(bma_errors))
    errors["post_mul"].append(np.array(post_mul_errors))
    errors["BOR_post_mul"].append(np.array(post_mul_BORs))
#     errors["Tbma_noLA"].append(np.array(Tbma_noLA_errors))
    errors["Tbma_LA"].append(np.array(Tbma_LA_errors))
#     errors["BOR_Tbma_noLA"].append(np.array(Tbma_noLA_BORs))
    errors["BOR_Tbma_LA"].append(np.array(Tbma_LA_BORs))
    
    print("BMA error rate for a cluster size: ",cluster_size,"is ",np.array(bma_errors).mean())
    print("Post multiply error rate for a cluster size: ",cluster_size,"is ",np.array(post_mul_errors).mean())
    print("Post multiply BORs for a cluster size: ",cluster_size,"is ",np.array(post_mul_BORs).mean())
#     print("Trellis BMA (no LA) error rate for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_errors).mean())
    print("Trellis BMA (with LA) error rate for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_errors).mean())
#     print("Trellis BMA (no LA) BORs for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_BORs).mean())
    print("Trellis BMA (with LA) BORs for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_BORs).mean())
    
    time.sleep(0.5)

errors["bma"] = np.array(errors["bma"])
errors["post_mul"] = np.array(errors["post_mul"])
errors["BOR_post_mul"] = np.array(errors["BOR_post_mul"])
errors["Tbma_noLA"] = np.array(errors["Tbma_noLA"])
errors["Tbma_LA"] = np.array(errors["Tbma_LA"])
errors["BOR_Tbma_noLA"] = np.array(errors["BOR_Tbma_noLA"])
errors["BOR_Tbma_LA"] = np.array(errors["BOR_Tbma_LA"])

# np.save("SavedData/sim_uncoded_rest.npy",errors)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:54<00:00,  1.83it/s]


BMA error rate for a cluster size:  2 is  42.31
Post multiply error rate for a cluster size:  2 is  32.13
Post multiply BORs for a cluster size:  2 is  1.0372628776986614
Trellis BMA (with LA) error rate for a cluster size: 2 is  41.61
Trellis BMA (with LA) BORs for a cluster size: 2 is  -0.7764993716583867


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:19<00:00,  1.25it/s]


BMA error rate for a cluster size:  3 is  26.07
Post multiply error rate for a cluster size:  3 is  31.63
Post multiply BORs for a cluster size:  3 is  0.9852509824288205
Trellis BMA (with LA) error rate for a cluster size: 3 is  14.68
Trellis BMA (with LA) BORs for a cluster size: 3 is  0.376105821325359


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:18<00:00,  2.58s/it]


BMA error rate for a cluster size:  10 is  0.59
Post multiply error rate for a cluster size:  10 is  25.32
Post multiply BORs for a cluster size:  10 is  0.6093447735310888
Trellis BMA (with LA) error rate for a cluster size: 10 is  0.0
Trellis BMA (with LA) BORs for a cluster size: 10 is  1.9998406321886992


In [ ]:
max_iters = 50

errors = {}
errors["desc"] = "Experiment comparing error rates and IRs on simulated data without coding. \n\
Algorithms compared: Improved BMALA, Post multiply, Trellis BMA no look-ahead, Trellis BMA with look-ahead.\n\
"

errors["bma"] = []
errors["post_mul"] = []
errors["BOR_post_mul"] = []
errors["Tbma_noLA"] = []
errors["Tbma_LA"] = []
errors["BOR_Tbma_noLA"] = []
errors["BOR_Tbma_LA"] = []
errors["cluster_size"] = [2,3,10]

for cluster_size in errors["cluster_size"]:
    bma_errors = []
    post_mul_errors = []
    post_mul_BORs = []
    Tbma_noLA_errors = []
    Tbma_LA_errors = []
    Tbma_noLA_BORs = []
    Tbma_LA_BORs = []
    
    for it in trange(max_iters):
#         if len(traces_list[it]) == 0:
#             print("Encountered empty cluster, ignored it.")
        
        in_seq = np.random.choice(4,size = in_len)
        
        tr_list = []
        for j in range(cluster_size):
            tr_list.append(ids_trellis.simulate_ids(in_seq))
            
        traces = []
        for tr in tr_list:
            if np.abs(len(tr)-N_cw) <= max_drift:
                traces.append(tr)
            elif len(tr) > N_cw:
                idx_delete = np.random.choice(len(tr),len(tr)-N_cw-max_drift, replace = False)
                traces.append(np.delete(tr,idx_delete))
            else:
                idx_insert = np.random.choice(len(tr),N_cw-max_drift-len(tr), replace = False)
                traces.append(np.insert(tr,idx_insert,0))
        
        bma_estimate = bmala_imp(N_cw,tr_list,2)
        
        post_mul_estimate, post_mul_probs = post_multiply(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1])
        
        Tbma_noLA_estimate, Tbma_noLA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1],lookahead = False)
        Tbma_LA_estimate, Tbma_LA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1],lookahead = True)
        
        bma_errors.append((in_seq != bma_estimate).sum())
        
        post_mul_errors.append((in_seq != post_mul_estimate).sum())
        post_mul_BORs.append(BCJROR(post_mul_probs,in_seq))
        
        Tbma_noLA_errors.append((in_seq != Tbma_noLA_estimate).sum())
        Tbma_LA_errors.append((in_seq != Tbma_LA_estimate).sum())
        Tbma_noLA_BORs.append(BCJROR(Tbma_noLA_probs, in_seq))
        Tbma_LA_BORs.append(BCJROR(Tbma_LA_probs, in_seq))

    errors["bma"].append(np.array(bma_errors))
    errors["post_mul"].append(np.array(post_mul_errors))
    errors["BOR_post_mul"].append(np.array(post_mul_BORs))
    errors["Tbma_noLA"].append(np.array(Tbma_noLA_errors))
    errors["Tbma_LA"].append(np.array(Tbma_LA_errors))
    errors["BOR_Tbma_noLA"].append(np.array(Tbma_noLA_BORs))
    errors["BOR_Tbma_LA"].append(np.array(Tbma_LA_BORs))
    
    print("BMA error rate for a cluster size: ",cluster_size,"is ",np.array(bma_errors).mean())
    print("Post multiply error rate for a cluster size: ",cluster_size,"is ",np.array(post_mul_errors).mean())
    print("Post multiply BORs for a cluster size: ",cluster_size,"is ",np.array(post_mul_BORs).mean())
    print("Trellis BMA (no LA) error rate for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_errors).mean())
    print("Trellis BMA (with LA) error rate for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_errors).mean())
    print("Trellis BMA (no LA) BORs for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_BORs).mean())
    print("Trellis BMA (with LA) BORs for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_BORs).mean())
    
    time.sleep(0.5)

errors["bma"] = np.array(errors["bma"])
errors["post_mul"] = np.array(errors["post_mul"])
errors["BOR_post_mul"] = np.array(errors["BOR_post_mul"])
errors["Tbma_noLA"] = np.array(errors["Tbma_noLA"])
errors["Tbma_LA"] = np.array(errors["Tbma_LA"])
errors["BOR_Tbma_noLA"] = np.array(errors["BOR_Tbma_noLA"])
errors["BOR_Tbma_LA"] = np.array(errors["BOR_Tbma_LA"])

# np.save("SavedData/sim_uncoded_rest.npy",errors)

In [ ]:
errors_multiD = np.load("SavedData/sim_uncoded_multiD.npy",allow_pickle = True).item()
errors_rest = np.load("SavedData/sim_uncoded_rest.npy",allow_pickle = True).item()

In [ ]:
import tikzplotlib

make_plot_template(plot_type = "Information rate", xlim = (1.5,10.5), ylim = (0,2.1))

keys = ["multiD_AIRs", "multiD_BORs"]
label = {}
label["multiD_AIRs"] = "AIRs for multiD"
label["multiD_BORs"] = "BCJR-OR for multiD"
add_plot(errors_multiD, 110, keys,  label, "Information rate")

keys = ["BOR_Tbma_LA", "BOR_Tbma_noLA","BOR_post_mul"]
label = {}
label["BOR_Tbma_noLA"] = "BCJR-OR Trellis BMA (no LA)"
label["BOR_Tbma_LA"] = "BCJR-OR Trellis BMA (LA)"
label["BOR_post_mul"] = "BCJR-OR Multiply posteriors"
add_plot(errors_rest, 110, keys, label, "Information rate")

keys = ["BOR_post_mul"]
add_plot(errors, 110, keys, label, "Information rate")

# tikzplotlib.save("Plots/sim_uncoded_IRs.tex")

In [ ]:
make_plot_template(plot_type = "Hamming rate", xlim = (1,11), ylim = (0,0.4))

keys = ["multiD"]
label = {}
label["multiD"] = "AIRs for multiD"
add_plot(errors_multiD, 110, keys,  label, "Hamming rate")


keys = ["bma","Tbma_LA", "Tbma_noLA","post_mul"]
label = {}
label["bma"] = "BMALA Improved"
label["Tbma_noLA"] = "Trellis BMA (no LA)"
label["Tbma_LA"] = "Trellis BMA (LA)"
label["post_mul"] = "Multiply posteriors"
add_plot(errors_rest, 110, keys,  label, "Hamming rate")

keys = ["post_mul"]
add_plot(errors, 110, keys, label, "Hamming rate")

tikzplotlib.save("Plots/sim_uncoded.tex")

## Coded error rates

In [ ]:
in_len = 55
N_cw = 110
redundancy = N_cw-in_len

A_in = 4
A_cw = 4

cc = conv_code()
G = np.array([[1,1,1],[1,2,1]])
cc.quar_cc(G)
cc.make_trellis(in_len)
cc.puncture(redundancy=redundancy)
cc.add_coset()
cc.make_encoder()
code_trellis_states = cc.trellis_states
code_trellis_edges = cc.trellis_edges
code_time_type = cc.time_type

num_traces = 1
p_del = 0.02      
p_sub = 0.022
p_ins = 0.017
max_drift = 6

ids_trellis = coded_ids_multiD(A_in, A_cw, code_trellis_states,code_trellis_edges, code_time_type,\
                 num_traces, p_del, p_sub, p_ins, max_drift, input_prior = None)

### MultiD

In [ ]:
# Real data
max_iters = 1000

errors = {}
errors["desc"] = "Experiment comparing error rates and IRs on simulated data without coding. \n\
Algorithms compared: MultiD bcjr.\n\
"

errors["multiD"] = []
errors["multiD_AIRs"] = []
errors["multiD_BORs"] = []
errors["cluster_size"] = [2,3]

for cluster_size in errors["cluster_size"]:
    if cluster_size == 2:
        max_drift = 10
    elif cluster_size == 3:
        max_drift = 6
    elif cluster_size == 4:
        max_drift = 4
    
    ids_trellis = coded_ids_multiD(A_in, A_cw, code_trellis_states,code_trellis_edges, code_time_type,\
                     cluster_size, p_del, p_sub, p_ins, max_drift, input_prior = None)

    multiD_errors = []
    multiD_AIRs = []
    multiD_BORs = []
    
    for it in trange(max_iters):
        in_seq = np.random.choice(4,size = in_len)
        coded_seq = cc.encode(in_seq)
        
        tr_list = []
        for j in range(cluster_size):
            tr_list.append(ids_trellis.simulate_ids(coded_seq))
            
        traces = []
        for tr in tr_list:
            if np.abs(len(tr)-N_cw) <= max_drift:
                traces.append(tr)
            elif len(tr) > N_cw:
                idx_delete = np.random.choice(len(tr),len(tr)-N_cw-max_drift, replace = False)
                traces.append(np.delete(tr,idx_delete))
            else:
                idx_insert = np.random.choice(len(tr),N_cw-max_drift-len(tr), replace = False)
                traces.append(np.insert(tr,idx_insert,0))
                
        post_probs = ids_trellis.bcjr(traces,cc.trellis_states[0][0],cc.trellis_states[-1])
        multiD_estimate = post_probs.argmax(axis=1)
        multiD_errors.append((multiD_estimate != in_seq).sum())
        
        logp_y = ids_trellis.logp_y * 1.0
        P = np.zeros((in_len,4))
        P[np.arange(in_len),in_seq] = 1
        ids_trellis.bcjr(traces,cc.trellis_states[0][0],cc.trellis_states[-1], input_prior = P)
        logp_ylx = ids_trellis.logp_y * 1.0
        
        if (logp_ylx-logp_y)/in_len == -np.inf:
            raise ValueError("Infinity")
        
        multiD_AIRs.append((logp_ylx-logp_y)/in_len)
        multiD_BORs.append(BCJROR(post_probs, in_seq))
        
        
    errors["multiD"].append(np.array(multiD_errors))
    errors["multiD_AIRs"].append(np.array(multiD_AIRs))
    errors["multiD_BORs"].append(np.array(multiD_BORs))
    
    print("MultiD error rate for a cluster size: ",cluster_size,"is ",np.array(multiD_errors).mean())
    print("MultiD AIRs for a cluster size: ",cluster_size,"is ",np.array(multiD_AIRs).mean())
    print("MultiD BORs for a cluster size: ",cluster_size,"is ",np.array(multiD_BORs).mean())
    
    time.sleep(0.5)

errors["multiD"] = np.array(errors["multiD"])
errors["multiD_AIRs"] = np.array(errors["multiD_AIRs"])
errors["multiD_BORs"] = np.array(errors["multiD_BORs"])

np.save("SavedData/sim_10coded_multiD.npy",errors)

### Trellis BMA and others

In [6]:
in_len = 100
N_cw = 110
redundancy = N_cw-in_len

A_in = 4
A_cw = 4

cc = conv_code()
G = np.array([[1],[1]])
cc.quar_cc(G)
cc.make_trellis(in_len)
cc.puncture(redundancy=redundancy)
cc.add_coset()
cc.make_encoder()
code_trellis_states = cc.trellis_states
code_trellis_edges = cc.trellis_edges
code_time_type = cc.time_type

num_traces = 1
p_del = 0.02      
p_sub = 0.022
p_ins = 0.017
max_drift = 15

ids_trellis = coded_ids_multiD(A_in, A_cw, code_trellis_states,code_trellis_edges, code_time_type,\
                 num_traces, p_del, p_sub, p_ins, max_drift, input_prior = None)

In [8]:
max_iters = 500

errors = {}
errors["desc"] = "Experiment comparing error rates and IRs on simulated data without coding. \n\
Algorithms compared: Improved BMALA, Post multiply, Trellis BMA no look-ahead, Trellis BMA with look-ahead.\n\
"

# errors["bma"] = []
errors["post_mul"] = []
errors["BOR_post_mul"] = []
errors["Tbma_noLA"] = []
errors["Tbma_LA"] = []
errors["BOR_Tbma_noLA"] = []
errors["BOR_Tbma_LA"] = []
errors["cluster_size"] = [2,3,6,10]

for cluster_size in errors["cluster_size"]:
    bma_errors = []
    post_mul_errors = []
    post_mul_BORs = []
    Tbma_noLA_errors = []
    Tbma_LA_errors = []
    Tbma_noLA_BORs = []
    Tbma_LA_BORs = []
    
    for it in trange(max_iters):
#         if len(traces_list[it]) == 0:
#             print("Encountered empty cluster, ignored it.")
        
        in_seq = np.random.choice(4,size = in_len)
        coded_seq = cc.encode(in_seq)
        
        tr_list = []
        for j in range(cluster_size):
            tr_list.append(ids_trellis.simulate_ids(coded_seq))
            
        traces = []
        for tr in tr_list:
            if np.abs(len(tr)-N_cw) <= max_drift:
                traces.append(tr)
            elif len(tr) > N_cw:
                idx_delete = np.random.choice(len(tr),len(tr)-N_cw-max_drift, replace = False)
                traces.append(np.delete(tr,idx_delete))
            else:
                idx_insert = np.random.choice(len(tr),N_cw-max_drift-len(tr), replace = False)
                traces.append(np.insert(tr,idx_insert,0))
        
#         bma_estimate = bmala_imp(N_cw,tr_list,2)
        
        post_mul_estimate, post_mul_probs = post_multiply(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1])
        
#         Tbma_noLA_estimate, Tbma_noLA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
#                                                  cc.trellis_states[-1],lookahead = False)
        Tbma_LA_estimate, Tbma_LA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1],lookahead = True)
        
#         bma_errors.append((in_seq != bma_estimate).sum())
        
        post_mul_errors.append((in_seq != post_mul_estimate).sum())
        post_mul_BORs.append(BCJROR(post_mul_probs,in_seq))
        
#         Tbma_noLA_errors.append((in_seq != Tbma_noLA_estimate).sum())
        Tbma_LA_errors.append((in_seq != Tbma_LA_estimate).sum())
#         Tbma_noLA_BORs.append(BCJROR(Tbma_noLA_probs, in_seq))
        Tbma_LA_BORs.append(BCJROR(Tbma_LA_probs, in_seq))

#     errors["bma"].append(np.array(bma_errors))
    errors["post_mul"].append(np.array(post_mul_errors))
    errors["BOR_post_mul"].append(np.array(post_mul_BORs))
#     errors["Tbma_noLA"].append(np.array(Tbma_noLA_errors))
    errors["Tbma_LA"].append(np.array(Tbma_LA_errors))
#     errors["BOR_Tbma_noLA"].append(np.array(Tbma_noLA_BORs))
    errors["BOR_Tbma_LA"].append(np.array(Tbma_LA_BORs))
    
#     print("BMA error rate for a cluster size: ",cluster_size,"is ",np.array(bma_errors).mean())
    print("Post multiply error rate for a cluster size: ",cluster_size,"is ",np.array(post_mul_errors).mean())
    print("Post multiply BORs for a cluster size: ",cluster_size,"is ",np.array(post_mul_BORs).mean())
#     print("Trellis BMA (no LA) error rate for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_errors).mean())
    print("Trellis BMA (with LA) error rate for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_errors).mean())
#     print("Trellis BMA (no LA) BORs for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_BORs).mean())
    print("Trellis BMA (with LA) BORs for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_BORs).mean())
    
    time.sleep(0.5)

# errors["bma"] = np.array(errors["bma"])
errors["post_mul"] = np.array(errors["post_mul"])
errors["BOR_post_mul"] = np.array(errors["BOR_post_mul"])
errors["Tbma_noLA"] = np.array(errors["Tbma_noLA"])
errors["Tbma_LA"] = np.array(errors["Tbma_LA"])
errors["BOR_Tbma_noLA"] = np.array(errors["BOR_Tbma_noLA"])
errors["BOR_Tbma_LA"] = np.array(errors["BOR_Tbma_LA"])

# np.save("SavedData/sim_6by11coded_rest.npy",errors)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [04:16<00:00,  1.95it/s]


Post multiply error rate for a cluster size:  2 is  8.814
Post multiply BORs for a cluster size:  2 is  1.6660736201776645
Trellis BMA (with LA) error rate for a cluster size: 2 is  8.84
Trellis BMA (with LA) BORs for a cluster size: 2 is  1.5243942879385026


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [06:20<00:00,  1.32it/s]


Post multiply error rate for a cluster size:  3 is  5.426
Post multiply BORs for a cluster size:  3 is  1.7881638707134497
Trellis BMA (with LA) error rate for a cluster size: 3 is  3.244
Trellis BMA (with LA) BORs for a cluster size: 3 is  1.7234379130042141


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [12:29<00:00,  1.50s/it]


Post multiply error rate for a cluster size:  6 is  2.22
Post multiply BORs for a cluster size:  6 is  1.9026512895715153
Trellis BMA (with LA) error rate for a cluster size: 6 is  0.326
Trellis BMA (with LA) BORs for a cluster size: 6 is  1.9431147124310324


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [20:45<00:00,  2.49s/it]


Post multiply error rate for a cluster size:  10 is  0.892
Post multiply BORs for a cluster size:  10 is  1.9571771638867526
Trellis BMA (with LA) error rate for a cluster size: 10 is  0.016
Trellis BMA (with LA) BORs for a cluster size: 10 is  1.9963860131899394


In [3]:
max_iters = 50

errors = {}
errors["desc"] = "Experiment comparing error rates and IRs on simulated data without coding. \n\
Algorithms compared: Improved BMALA, Post multiply, Trellis BMA no look-ahead, Trellis BMA with look-ahead.\n\
"

# errors["bma"] = []
errors["post_mul"] = []
errors["BOR_post_mul"] = []
errors["Tbma_noLA"] = []
errors["Tbma_LA"] = []
errors["BOR_Tbma_noLA"] = []
errors["BOR_Tbma_LA"] = []
errors["cluster_size"] = [2,3,6,10]

for cluster_size in errors["cluster_size"]:
    bma_errors = []
    post_mul_errors = []
    post_mul_BORs = []
    Tbma_noLA_errors = []
    Tbma_LA_errors = []
    Tbma_noLA_BORs = []
    Tbma_LA_BORs = []
    
    for it in trange(max_iters):
#         if len(traces_list[it]) == 0:
#             print("Encountered empty cluster, ignored it.")
        
        in_seq = np.random.choice(4,size = in_len)
        coded_seq = cc.encode(in_seq)
        
        tr_list = []
        for j in range(cluster_size):
            tr_list.append(ids_trellis.simulate_ids(coded_seq))
            
        traces = []
        for tr in tr_list:
            if np.abs(len(tr)-N_cw) <= max_drift:
                traces.append(tr)
            elif len(tr) > N_cw:
                idx_delete = np.random.choice(len(tr),len(tr)-N_cw-max_drift, replace = False)
                traces.append(np.delete(tr,idx_delete))
            else:
                idx_insert = np.random.choice(len(tr),N_cw-max_drift-len(tr), replace = False)
                traces.append(np.insert(tr,idx_insert,0))
        
#         bma_estimate = bmala_imp(N_cw,tr_list,2)
        
        post_mul_estimate, post_mul_probs = post_multiply(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1])
        
        Tbma_noLA_estimate, Tbma_noLA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1],lookahead = False)
        Tbma_LA_estimate, Tbma_LA_probs = trellis_bma(ids_trellis,traces,cc.trellis_states[0][0],\
                                                 cc.trellis_states[-1],lookahead = True)
        
#         bma_errors.append((in_seq != bma_estimate).sum())
        
        post_mul_errors.append((in_seq != post_mul_estimate).sum())
        post_mul_BORs.append(BCJROR(post_mul_probs,in_seq))
        
        Tbma_noLA_errors.append((in_seq != Tbma_noLA_estimate).sum())
        Tbma_LA_errors.append((in_seq != Tbma_LA_estimate).sum())
        Tbma_noLA_BORs.append(BCJROR(Tbma_noLA_probs, in_seq))
        Tbma_LA_BORs.append(BCJROR(Tbma_LA_probs, in_seq))

#     errors["bma"].append(np.array(bma_errors))
    errors["post_mul"].append(np.array(post_mul_errors))
    errors["BOR_post_mul"].append(np.array(post_mul_BORs))
    errors["Tbma_noLA"].append(np.array(Tbma_noLA_errors))
    errors["Tbma_LA"].append(np.array(Tbma_LA_errors))
    errors["BOR_Tbma_noLA"].append(np.array(Tbma_noLA_BORs))
    errors["BOR_Tbma_LA"].append(np.array(Tbma_LA_BORs))
    
#     print("BMA error rate for a cluster size: ",cluster_size,"is ",np.array(bma_errors).mean())
    print("Post multiply error rate for a cluster size: ",cluster_size,"is ",np.array(post_mul_errors).mean())
    print("Post multiply BORs for a cluster size: ",cluster_size,"is ",np.array(post_mul_BORs).mean())
    print("Trellis BMA (no LA) error rate for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_errors).mean())
    print("Trellis BMA (with LA) error rate for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_errors).mean())
    print("Trellis BMA (no LA) BORs for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_BORs).mean())
    print("Trellis BMA (with LA) BORs for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_BORs).mean())
    
    time.sleep(0.5)

# errors["bma"] = np.array(errors["bma"])
errors["post_mul"] = np.array(errors["post_mul"])
errors["BOR_post_mul"] = np.array(errors["BOR_post_mul"])
errors["Tbma_noLA"] = np.array(errors["Tbma_noLA"])
errors["Tbma_LA"] = np.array(errors["Tbma_LA"])
errors["BOR_Tbma_noLA"] = np.array(errors["BOR_Tbma_noLA"])
errors["BOR_Tbma_LA"] = np.array(errors["BOR_Tbma_LA"])

# np.save("SavedData/sim_6by11coded_rest.npy",errors)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:13<00:00,  1.36it/s]


Post multiply error rate for a cluster size:  2 is  9.4
Post multiply BORs for a cluster size:  2 is  1.6449181138118696
Trellis BMA (no LA) error rate for a cluster size:  2 is  30.43
Trellis BMA (with LA) error rate for a cluster size: 2 is  9.95
Trellis BMA (no LA) BORs for a cluster size:  2 is  -0.07988426271437732
Trellis BMA (with LA) BORs for a cluster size: 2 is  1.4128116571218032


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:30<00:00,  1.11it/s]


Post multiply error rate for a cluster size:  3 is  5.85
Post multiply BORs for a cluster size:  3 is  1.7747916911664838
Trellis BMA (no LA) error rate for a cluster size:  3 is  9.75
Trellis BMA (with LA) error rate for a cluster size: 3 is  4.29
Trellis BMA (no LA) BORs for a cluster size:  3 is  1.0520339292686505
Trellis BMA (with LA) BORs for a cluster size: 3 is  1.6228515245539503


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:56<00:00,  1.77s/it]


Post multiply error rate for a cluster size:  6 is  2.35
Post multiply BORs for a cluster size:  6 is  1.8960955891124174
Trellis BMA (no LA) error rate for a cluster size:  6 is  1.69
Trellis BMA (with LA) error rate for a cluster size: 6 is  0.1
Trellis BMA (no LA) BORs for a cluster size:  6 is  1.703111794364914
Trellis BMA (with LA) BORs for a cluster size: 6 is  1.9898272641539358


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:54<00:00,  2.94s/it]


Post multiply error rate for a cluster size:  10 is  1.24
Post multiply BORs for a cluster size:  10 is  1.9372324979046984
Trellis BMA (no LA) error rate for a cluster size:  10 is  0.08
Trellis BMA (with LA) error rate for a cluster size: 10 is  0.39
Trellis BMA (no LA) BORs for a cluster size:  10 is  1.9904491195410259
Trellis BMA (with LA) BORs for a cluster size: 10 is  1.8871739306017203


In [4]:
errors["BOR_Tbma_LA"]

array([[ 1.49689572,  1.80048353,  1.89266779,  0.74791203,  1.40063531,
         1.9634951 ,  0.99793395,  1.96688495,  1.63122779,  1.95020473,
         0.02457578,  1.43026946,  0.50048059,  1.5346318 ,  1.94318597,
         1.83634538,  1.53249395,  1.8220799 ,  1.2276679 ,  0.60521188,
        -0.81594501,  0.76207164,  1.78948615,  1.19694865,  0.88564428,
         1.53873581,  0.63958403,  0.27538666,  1.92818848,  1.60097374,
         1.39846652,  1.92791298,  1.95556708,  1.95174396,  1.31017884,
         1.92560811,  1.5244865 ,  0.8304638 ,  1.55834623,  1.93865056,
         1.80097552,  1.78055886,  1.94492423,  1.95753627,  1.62521633,
         1.0616323 ,  1.87544524,  1.87864174,  1.31235834,  0.28254847,
         1.92321498,  1.43798349,  1.88207365,  0.85181899,  1.01349355,
         0.45478011,  0.52673355,  1.73991538,  1.89979569,  1.94782691,
         1.83188049,  1.55302794,  1.60430958,  1.93572664,  1.89507135,
         1.9238106 ,  1.88179406,  1.85566121,  1.8

In [ ]:
errors_multiD = np.load("SavedData/sim_10coded_multiD.npy",allow_pickle = True).item()
errors_rest = np.load("SavedData/sim_10coded_rest.npy",allow_pickle = True).item()

In [ ]:
make_plot_template(plot_type = "Information rate", xlim = (1.5,10.5), ylim = (0,2.1))

keys = ["multiD_AIRs", "multiD_BORs"]
label = {}
label["multiD_AIRs"] = "AIRs for multiD"
label["multiD_BORs"] = "BCJR-OR for multiD"
add_plot(errors_multiD, 110, keys,  label, "Information rate")

keys = ["BOR_Tbma_LA", "BOR_Tbma_noLA","BOR_post_mul"]
label = {}
label["BOR_Tbma_noLA"] = "BCJR-OR Trellis BMA (no LA)"
label["BOR_Tbma_LA"] = "BCJR-OR Trellis BMA (LA)"
label["BOR_post_mul"] = "BCJR-OR Multiply posteriors"
add_plot(errors_rest, 110, keys, label, "Information rate")

tikzplotlib.save("Plots/sim_10coded_IRs.tex")

make_plot_template(plot_type = "Hamming rate", xlim = (1,11), ylim = (0,0.3))

keys = ["multiD"]
label = {}
label["multiD"] = "MultiD"
add_plot(errors_multiD, 110, keys,  label, "Hamming rate")


keys = ["Tbma_LA", "Tbma_noLA","post_mul"]
label = {}
label["bma"] = "BMALA Improved"
label["Tbma_noLA"] = "Trellis BMA (no LA)"
label["Tbma_LA"] = "Trellis BMA (LA)"
label["post_mul"] = "Multiply posteriors"
add_plot(errors_rest, 110, keys,  label, "Hamming rate")

tikzplotlib.save("Plots/sim_10coded.tex")

## Multiprocessing attempt

In [ ]:
max_iters = 160

process_per_hyperiter = 16
hyperiters = max_iters//process_per_hyperiter
max_iters = process_per_hyperiter*hyperiters

errors = {}
errors["desc"] = "Experiment comparing error rates and IRs on simulated data without coding. \n\
Algorithms compared: Improved BMALA, Post multiply, Trellis BMA no look-ahead, Trellis BMA with look-ahead.\n\
"

errors["bma"] = []
errors["post_mul"] = []
errors["BOR_post_mul"] = []
errors["Tbma_noLA"] = []
errors["Tbma_LA"] = []
errors["BOR_Tbma_noLA"] = []
errors["BOR_Tbma_LA"] = []
errors["cluster_size"] = [10]

for cluster_size in errors["cluster_size"]:
    bma_errors = []
    post_mul_errors = []
    post_mul_BORs = []
    Tbma_noLA_errors = []
    Tbma_LA_errors = []
    Tbma_noLA_BORs = []
    Tbma_LA_BORs = []
    
    pool1 = Pool(mp.cpu_count())
    for it in trange(hyperiters):
        
        in_seq_list = []
        tr_list_list = []
        for j in range(process_per_hyperiter):
            in_seq = np.random.choice(4,size = in_len)
            in_seq_list.append(in_seq)
            tr_list = []
            for j in range(cluster_size):
                tr_list.append(ids_trellis.simulate_ids(in_seq))
            tr_list_list.append(tr_list)
        
        
        temp = pool1.starmap(one_iter_rest, zip(repeat(in_len), in_seq_list, tr_list_list,\
                                          repeat(cluster_size), repeat(N_cw), repeat(ids_trellis), repeat(cc)))
    pool1.close()
#         temp = np.array(temp)
#         hamming_error_list[idx,:] += temp.sum(axis = 0)
#         if len(traces_list[it]) == 0:
#             print("Encountered empty cluster, ignored it.")
        
        
#         (bma_error_, post_mul_error_, Tbma_noLA_error_, Tbma_LA_error_,\
#            post_mul_BOR_,Tbma_noLA_BOR_,Tbma_LA_BOR_) = pool.starmap(one_iter)
        
#         bma_errors.append((in_seq != bma_estimate).sum())
        
#         post_mul_errors.append((in_seq != post_mul_estimate).sum())
#         post_mul_BORs.append(BCJROR(post_mul_probs,in_seq))
        
#         Tbma_noLA_errors.append((in_seq != Tbma_noLA_estimate).sum())
#         Tbma_LA_errors.append((in_seq != Tbma_LA_estimate).sum())
#         Tbma_noLA_BORs.append(BCJROR(Tbma_noLA_probs, in_seq))
#         Tbma_LA_BORs.append(BCJROR(Tbma_LA_probs, in_seq))
    
    
#     errors["bma"].append(np.array(bma_errors))
#     errors["post_mul"].append(np.array(post_mul_errors))
#     errors["BOR_post_mul"].append(np.array(post_mul_BORs))
#     errors["Tbma_noLA"].append(np.array(Tbma_noLA_errors))
#     errors["Tbma_LA"].append(np.array(Tbma_LA_errors))
#     errors["BOR_Tbma_noLA"].append(np.array(Tbma_noLA_BORs))
#     errors["BOR_Tbma_LA"].append(np.array(Tbma_LA_BORs))
    
#     print("BMA error rate for a cluster size: ",cluster_size,"is ",np.array(bma_errors).mean())
#     print("Post multiply error rate for a cluster size: ",cluster_size,"is ",np.array(post_mul_errors).mean())
#     print("Post multiply BORs for a cluster size: ",cluster_size,"is ",np.array(post_mul_BORs).mean())
#     print("Trellis BMA (no LA) error rate for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_errors).mean())
#     print("Trellis BMA (with LA) error rate for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_errors).mean())
#     print("Trellis BMA (no LA) BORs for a cluster size: ",cluster_size,"is ",np.array(Tbma_noLA_BORs).mean())
#     print("Trellis BMA (with LA) BORs for a cluster size:",cluster_size,"is ",np.array(Tbma_LA_BORs).mean())
    
    time.sleep(0.5)

errors["bma"] = np.array(errors["bma"])
errors["post_mul"] = np.array(errors["post_mul"])
errors["BOR_post_mul"] = np.array(errors["BOR_post_mul"])
errors["Tbma_noLA"] = np.array(errors["Tbma_noLA"])
errors["Tbma_LA"] = np.array(errors["Tbma_LA"])
errors["BOR_Tbma_noLA"] = np.array(errors["BOR_Tbma_noLA"])
errors["BOR_Tbma_LA"] = np.array(errors["BOR_Tbma_LA"])

# np.save("SavedData/sim_uncoded_rest.npy",errors)